# Notebook for replicating the analyses in "How does a Move to a Flat Tax Affect Household Filers? The Case of Kansas"
### Richard W. Evans and Patrick Neyland, June 2023
This notebook replicates the analyses in the *Research in Focus* article by [Richard W. Evans](https://sites.google.com/site/rickecon) and Patrick Neyland entitled "How does a Move to a Flat Tax Affect Household Filers? The Case of Kansas".

## 1. Introduction
Before opening and running this notebook, make sure that you have downloaded or cloned the [`KS-FlatTax`](https://github.com/TheCGO/KS-FlatTax) repository (https://github.com/TheCGO/KS-FlatTax) and created and activated the associated conda environment `ks-flattax-dev` in the `environment.yml` file. You can also run this notebook easily in the cloud by going to this [Google Colab version](https://colab.research.google.com/drive/1phIZ1oJNs-IjRv7Av7uAHIX3T6P6XVQX?usp=sharing) of this notebook.

The notebook for the introduction produces the following figures.
* 1.1. Figure 1. Plot of US states by state employment income tax type
* 1.2. Figure 2. Rainy Day Fund and Total Reserves as Percentages of General-Fund Expenditures, Kansas and 50-State Median: 2000-2022
* 1.3. Figure 3. Estimated 2022 Rainy Day Fund Balances and Total Reserves and Balances as Percentages of General-Fund Expenditures

In [1]:
# Import packages
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.io import output_file, output_notebook, export_png
from bokeh.plotting import figure, show
from bokeh.models import (ColumnDataSource, Title, Label, LabelSet, Legend,
                          LegendItem, CategoricalColorMapper, ColorBar,
                          HoverTool, NumeralTickFormatter, GeoJSONDataSource)
from bokeh.models.tickers import SingleIntervalTicker
from bokeh.sampledata.us_states import data as states

import random
import json

from fiscalsim_us import Simulation
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant

### 1.1. Figure 1. Plot of US states by state employment income tax type

Create the data for state tax type.

In [3]:
# Create a DataFrame of states and their tax types as of January 1, 2023
# 0 = No state income tax
# 1 = Flat state income tax rate
# 2 = Nearly flat state income tax rates
# 3 = Progressive state income tax rates
state_taxtype_list = [
    ["Alabama", "AL", 2],
    ["Alaska", "AK", 0],
    ["Arizona", "AZ", 1],
    ["Arkansas", "AR", 3],
    ["California", "CA", 3],
    ["Colorado", "CO", 1],
    ["Connecticut", "CT", 3],
    ["Delaware", "DE", 3],
    ["District of Columbia", "DC", 3],
    ["Florida", "FL", 0],
    ["Georgia", "GA", 2],
    ["Hawaii", "HI", 3],
    ["Idaho", "ID", 1],
    ["Illinois", "IL", 1],
    ["Indiana", "IN", 1],
    ["Iowa", "IA", 2],
    ["Kansas", "KS", 3],
    ["Kentucky", "KY", 1],
    ["Louisiana", "LA", 3],
    ["Maine", "ME", 3],
    ["Maryland", "MD", 3],
    ["Massachusetts", "MA", 1],
    ["Michigan", "MI", 1],
    ["Minnesota", "MN", 3],
    ["Mississippi", "MS", 1],
    ["Missouri", "MO", 2],
    ["Montana", "MT", 3],
    ["Nebraska", "NE", 3],
    ["Nevada", "NV", 0],
    ["New Hampshire", "NH", 0],
    ["New Jersey", "NJ", 3],
    ["New Mexico", "NM", 3],
    ["New York", "NY", 3],
    ["North Carolina", "NC", 1],
    ["North Dakota", "ND", 3],
    ["Ohio", "OH", 3],
    ["Oklahoma", "OK", 2],
    ["Oregon", "OR", 3],
    ["Pennsylvania", "PA", 1],
    ["Rhode Island", "RI", 3],
    ["South Carolina", "SC", 3],
    ["South Dakota", "SD", 0],
    ["Tennessee", "TN", 0],
    ["Texas", "TX", 0],
    ["Utah", "UT", 1],
    ["Vermont", "VT", 3],
    ["Virginia", "VA", 3],
    ["Washington", "WA", 0],
    ["West Virginia", "WV", 3], 
    ["Wisconsin", "WI", 3],
    ["Wyoming", "WY", 0],
]
state_taxtype_df = pd.DataFrame(
    state_taxtype_list, columns=["State", "Abbrev", "TaxType"]
)

# define state rates

state_rates_2023 = [
    ["Alabama", '2.00%', '4.00%', '5.00%'],
    ["Alaska", '0.00%'],
    ["Arizona", '2.50%'],
    ["Arkansas", '2.00%', '4.00%', '4.90%'],
    ["California", '1.00%', '2.00%', '4.00%', '6.00%', '8.00%',
    '9.30%', '10.30%', '11.30%', '12.30%', '13.30%'],
    ["Colorado", '4.40%'],
    ["Connecticut", '3.00%', '5.00%', '5.50%', '6.00%', '6.50%', '6.90%', '6.99%'],
    ["Delaware", '2.20%', '3.90%', '4.80%', '5.20%', '5.55%', '6.60%'],
    ["District of Columbia", '4.00%', '6.00%', '6.50%','8.50%', '9.25%', '9.75%', '10.75%'],
    ["Florida", '0.00%'],
    ["Georgia", '1.00%', '2.00%', '3.00%', '4.00%', '5.00%', '5.75%'],
    ["Hawaii", '1.40%', '3.20%', '5.50%', '6.40%', '6.80%',	
    '7.20%','7.60%','7.90%','8.25%','9.00%','10.00%','11.00%'],
    ["Idaho", '5.80%'],
    ["Illinois", '4.95%'],
    ["Indiana", '3.15%'],
    ["Iowa", '4.40%', '4.82%', '5.70%', '6.00%'],
    ["Kansas", '3.10%', '5.25%', '5.70%'],
    ["Kentucky", '4.50%'],
    ["Louisiana", '1.85%', '3.50%', '4.25%'],
    ["Maine", '5.80%', '6.75%', '7.15%'],
    ["Maryland", '2.00%', '3.00%', '4.00%', '4.75%', '5.00%', '5.25%', '5.50%', '5.75%'],
    ["Massachusetts", '5.00%', '9.00%'],
    ["Michigan", '4.25%'],
    ["Minnesota", '5.35%', '6.80%', '7.85%', '9.85%'],
    ["Mississippi", '5.00%'],
    ["Missouri", '2.00%', '2.50%', '3.00%', '3.50%', '4.00%', '4.50%', '4.95%'],
    ["Montana", '1.00%', '2.00%', '3.00%', '4.00%', '5.00%', '6.00%', '6.75%'],
    ["Nebraska", '2.46%', '3.51%', '5.01%', '6.64%'],
    ["Nevada",'0.00%'],
    ["New Hampshire",'0.00%'],
    ["New Jersey",'1.400%', '1.750%', '3.500%', '5.525%', '6.370%', '8.970%', '10.750%'],
    ["New Mexico",'1.70%', '3.20%', '4.70%', '4.90%', '5.90%'],
    ["New York",'4.00%', '4.50%', '5.25%', '5.50%', '6.00%', '6.85%', '9.65%', '10.30%', '10.90%'],
    ["North Carolina",'4.75%'],
    ["North Dakota", '1.10%', '2.04%', '2.27%', '2.64%', '2.90%'],
    ["Ohio",'2.765%', '3.226%', '3.688%', '3.990%'],
    ["Oklahoma",'0.25%', '0.75%', '1.75%', '2.75%', '3.75%', '4.75%'],
    ["Oregon",'4.75%', '6.75%', '8.75%', '9.90%'],
    ["Pennsylvania",'3.07%'],
    ["Rhode Island",'3.75%', '4.75%', '5.99%'],
    ["South Carolina",'0.00%', '3.00%', '6.50%'],
    ["South Dakota",'0.00%'],
    ["Tennessee",'0.00%'],
    ["Texas",'0.00%'],
    ["Utah",'4.85%'],
    ["Vermont",'3.35%', '6.60%', '7.60%', '8.75%'],
    ["Virginia",'2.00%', '3.00%', '5.00%', '5.75%'],
    ["Washington",'0.00%'],
    ["West Virginia",'3.00%', '4.00%', '4.50%', '6.00%', '6.50%'], 
    ["Wisconsin",'3.54%', '4.65%', '5.30%', '7.65%'],
    ["Wyoming",'0.00%'],
]

#Remove the state name from the list
state_rates_2023 = [lst[1:] for lst in state_rates_2023]

state_taxtype_df['rates'] = state_rates_2023




state_taxtype_df["TaxType_str"] = ""
state_taxtype_df["TaxType_str"][state_taxtype_df["TaxType"]==0] = \
    "No state labor income tax"
state_taxtype_df["TaxType_str"][state_taxtype_df["TaxType"]==1] = \
    "Flat state income tax rate"
state_taxtype_df["TaxType_str"][state_taxtype_df["TaxType"]==2] = \
    "Nearly flat state income tax rates"
state_taxtype_df["TaxType_str"][state_taxtype_df["TaxType"]==3] = \
    "Progressive state income tax rates"
state_taxtype_df["TaxType_str_short"] = ""
state_taxtype_df["TaxType_str_short"][state_taxtype_df["TaxType"]==0] = \
    "No income tax"
state_taxtype_df["TaxType_str_short"][state_taxtype_df["TaxType"]==1] = \
    "Flat income tax rate"
state_taxtype_df["TaxType_str_short"][state_taxtype_df["TaxType"]==2] = \
    "Nearly flat income tax rates"
state_taxtype_df["TaxType_str_short"][state_taxtype_df["TaxType"]==3] = \
    "Progressive income tax rates"
state_taxtype_df["fill_color"] = ""
state_taxtype_df["fill_color"][state_taxtype_df["TaxType"]==0] = "white"
state_taxtype_df["fill_color"][state_taxtype_df["TaxType"]==1] = "red"
state_taxtype_df["fill_color"][state_taxtype_df["TaxType"]==2] = "purple"
state_taxtype_df["fill_color"][state_taxtype_df["TaxType"]==3] = "blue"

# Sort alphabetically by full state name
state_taxtype_df.sort_values("State", inplace=True, ignore_index=True)

# Create sub-DataFrames for each tax type
no_tax_states_df = \
    state_taxtype_df[state_taxtype_df["TaxType"]==0].reset_index(drop=True)
flat_tax_states_df = \
    state_taxtype_df[state_taxtype_df["TaxType"]==1].reset_index(drop=True)
nflat_tax_states_df = \
    state_taxtype_df[state_taxtype_df["TaxType"]==2].reset_index(drop=True)
prog_tax_states_df = \
    state_taxtype_df[state_taxtype_df["TaxType"]==3].reset_index(drop=True)
print(state_taxtype_df.groupby("TaxType").count())

# Save state_data_df as .csv
state_taxtype_df.to_csv('./data/state_taxtype.csv', index=False)
state_taxtype_df

         State  Abbrev  rates  TaxType_str  TaxType_str_short  fill_color
TaxType                                                                  
0            9       9      9            9                  9           9
1           12      12     12           12                 12          12
2            5       5      5            5                  5           5
3           25      25     25           25                 25          25


,State,Abbrev,TaxType,rates,TaxType_str,TaxType_str_short,fill_color
0,Alabama,AL,2,"[2.00%, 4.00%, 5.00%]",Nearly flat state income tax rates,Nearly flat income tax rates,purple
1,Alaska,AK,0,[0.00%],No state labor income tax,No income tax,white
2,Arizona,AZ,1,[2.50%],Flat state income tax rate,Flat income tax rate,red
3,Arkansas,AR,3,"[2.00%, 4.00%, 4.90%]",Progressive state income tax rates,Progressive income tax rates,blue
4,California,CA,3,"[1.00%, 2.00%, 4.00%, 6.00%, 8.00%, 9.30%, 10....",Progressive state income tax rates,Progressive income tax rates,blue
5,Colorado,CO,1,[4.40%],Flat state income tax rate,Flat income tax rate,red
6,Connecticut,CT,3,"[3.00%, 5.00%, 5.50%, 6.00%, 6.50%, 6.90%, 6.99%]",Progressive state income tax rates,Progressive income tax rates,blue
7,Delaware,DE,3,"[2.20%, 3.90%, 4.80%, 5.20%, 5.55%, 6.60%]",Progressive state income tax rates,Progressive income tax rates,blue
8,District of Columbia,DC,3,"[4.00%, 6.00%, 6.50%, 8.50%, 9.25%, 9.75%, 10....",Progressive state income tax rates,Progressive income tax rates,blue
9,Florida,FL,0,[0.00%],No state labor income tax,No income tax,white


In [4]:
# Print the no tax states
no_tax_states_df

,State,Abbrev,TaxType,rates,TaxType_str,TaxType_str_short,fill_color
0,Alaska,AK,0,[0.00%],No state labor income tax,No income tax,white
1,Florida,FL,0,[0.00%],No state labor income tax,No income tax,white
2,Nevada,NV,0,[0.00%],No state labor income tax,No income tax,white
3,New Hampshire,NH,0,[0.00%],No state labor income tax,No income tax,white
4,South Dakota,SD,0,[0.00%],No state labor income tax,No income tax,white
5,Tennessee,TN,0,[0.00%],No state labor income tax,No income tax,white
6,Texas,TX,0,[0.00%],No state labor income tax,No income tax,white
7,Washington,WA,0,[0.00%],No state labor income tax,No income tax,white
8,Wyoming,WY,0,[0.00%],No state labor income tax,No income tax,white


In [5]:
# Print the flat tax states
flat_tax_states_df

,State,Abbrev,TaxType,rates,TaxType_str,TaxType_str_short,fill_color
0,Arizona,AZ,1,[2.50%],Flat state income tax rate,Flat income tax rate,red
1,Colorado,CO,1,[4.40%],Flat state income tax rate,Flat income tax rate,red
2,Idaho,ID,1,[5.80%],Flat state income tax rate,Flat income tax rate,red
3,Illinois,IL,1,[4.95%],Flat state income tax rate,Flat income tax rate,red
4,Indiana,IN,1,[3.15%],Flat state income tax rate,Flat income tax rate,red
5,Kentucky,KY,1,[4.50%],Flat state income tax rate,Flat income tax rate,red
6,Massachusetts,MA,1,"[5.00%, 9.00%]",Flat state income tax rate,Flat income tax rate,red
7,Michigan,MI,1,[4.25%],Flat state income tax rate,Flat income tax rate,red
8,Mississippi,MS,1,[5.00%],Flat state income tax rate,Flat income tax rate,red
9,North Carolina,NC,1,[4.75%],Flat state income tax rate,Flat income tax rate,red


In [6]:
# Print the nearly flat tax states
nflat_tax_states_df

,State,Abbrev,TaxType,rates,TaxType_str,TaxType_str_short,fill_color
0,Alabama,AL,2,"[2.00%, 4.00%, 5.00%]",Nearly flat state income tax rates,Nearly flat income tax rates,purple
1,Georgia,GA,2,"[1.00%, 2.00%, 3.00%, 4.00%, 5.00%, 5.75%]",Nearly flat state income tax rates,Nearly flat income tax rates,purple
2,Iowa,IA,2,"[4.40%, 4.82%, 5.70%, 6.00%]",Nearly flat state income tax rates,Nearly flat income tax rates,purple
3,Missouri,MO,2,"[2.00%, 2.50%, 3.00%, 3.50%, 4.00%, 4.50%, 4.95%]",Nearly flat state income tax rates,Nearly flat income tax rates,purple
4,Oklahoma,OK,2,"[0.25%, 0.75%, 1.75%, 2.75%, 3.75%, 4.75%]",Nearly flat state income tax rates,Nearly flat income tax rates,purple


In [7]:
# Print the progressive tax states
prog_tax_states_df

,State,Abbrev,TaxType,rates,TaxType_str,TaxType_str_short,fill_color
0,Arkansas,AR,3,"[2.00%, 4.00%, 4.90%]",Progressive state income tax rates,Progressive income tax rates,blue
1,California,CA,3,"[1.00%, 2.00%, 4.00%, 6.00%, 8.00%, 9.30%, 10....",Progressive state income tax rates,Progressive income tax rates,blue
2,Connecticut,CT,3,"[3.00%, 5.00%, 5.50%, 6.00%, 6.50%, 6.90%, 6.99%]",Progressive state income tax rates,Progressive income tax rates,blue
3,Delaware,DE,3,"[2.20%, 3.90%, 4.80%, 5.20%, 5.55%, 6.60%]",Progressive state income tax rates,Progressive income tax rates,blue
4,District of Columbia,DC,3,"[4.00%, 6.00%, 6.50%, 8.50%, 9.25%, 9.75%, 10....",Progressive state income tax rates,Progressive income tax rates,blue
5,Hawaii,HI,3,"[1.40%, 3.20%, 5.50%, 6.40%, 6.80%, 7.20%, 7.6...",Progressive state income tax rates,Progressive income tax rates,blue
6,Kansas,KS,3,"[3.10%, 5.25%, 5.70%]",Progressive state income tax rates,Progressive income tax rates,blue
7,Louisiana,LA,3,"[1.85%, 3.50%, 4.25%]",Progressive state income tax rates,Progressive income tax rates,blue
8,Maine,ME,3,"[5.80%, 6.75%, 7.15%]",Progressive state income tax rates,Progressive income tax rates,blue
9,Maryland,MD,3,"[2.00%, 3.00%, 4.00%, 4.75%, 5.00%, 5.25%, 5.5...",Progressive state income tax rates,Progressive income tax rates,blue


Create the Bokeh state map figure

In [12]:
fig1_title = "Type of state employment income tax system as of January 2023"
output_file(
    "./images/state_taxtype_2023.html", title=fig1_title, mode='inline'
)
output_notebook()

# Download U.S. states shape files from US Census Bureau
# https://www.census.gov/geographies/mapping-files/2018/geo/carto-boundary-file.html
us_shapefile_path = ("https://github.com/TheCGO/KS-FlatTax/raw/main/data/" +
                     "cb_2018_us_state_20m/cb_2018_us_state_20m.shp")
gdf = gpd.GeoDataFrame.from_file(us_shapefile_path)
gdf_json = gdf.to_json()
gjson = json.loads(gdf_json)

# Remove Puerto Rico from data
del(gjson["features"][7])

# Alaska
# Fix positive longitudes
min_lat_ak = 180  # initial value that will be adjusted
min_abs_lon_ak = 180  # initial value that will be adjusted
coords_list = gjson["features"][24]["geometry"]["coordinates"]
for ind_isl, island in enumerate(coords_list):
    for ind_pnt, point in enumerate(island[0]):
        min_lat_ak = np.minimum(min_lat_ak, point[1])
        if point[0] > 0:
            gjson["features"][24]["geometry"][
                "coordinates"
            ][ind_isl][0][ind_pnt][0] = -180 - (180 - point[0])
        else:
            min_abs_lon_ak = np.minimum(min_abs_lon_ak, -point[0])

# Shrink the size of Alaska relative to its southestern most minimum lattitude
# and longitude
shrink_pct_ak = 0.65
coords_list_ak = gjson["features"][24]["geometry"]["coordinates"]
for ind_isl, island in enumerate(coords_list_ak):
    for ind_pnt, point in enumerate(island[0]):
        gjson["features"][24]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][0] = point[0] - shrink_pct_ak * (point[0] +
                                                                min_abs_lon_ak)
        gjson["features"][24]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][1] = point[1] - shrink_pct_ak * (point[1] -
                                                                min_lat_ak)

# Move Alaska closer to the mainland such that the minimum minimum absolute
# longitude and lattitude are (-127, 44)
min_lat_ak_new = 44
min_abs_lon_ak_new = 127
for ind_isl, island in enumerate(coords_list):
    for ind_pnt, point in enumerate(island[0]):
        gjson["features"][24]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][0] = point[0] + (min_abs_lon_ak -
                                                min_abs_lon_ak_new)
        gjson["features"][24]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][1] = point[1] - (min_lat_ak - min_lat_ak_new)

# Hawaii
list_ind_hi = 47
# Get minimum lattitude and minimum absolute longitude for Hawaii
min_lat_hi = 180  # initial value that will be adjusted
min_abs_lon_hi = 180  # initial value that will be adjusted
coords_list = gjson["features"][list_ind_hi]["geometry"]["coordinates"]
for ind_isl, island in enumerate(coords_list):
    for ind_pnt, point in enumerate(island[0]):
        min_lat_hi = np.minimum(min_lat_hi, point[1])
        min_abs_lon_hi = np.minimum(min_abs_lon_hi, -point[0])
# print("Minimum lattitude for Hawaii is", min_lat_hi)
# print("Minimum absolute longitude for Hawaii is", min_abs_lon_hi)

# Increase the size of Hawaii
incr_pct_hi = 0.4
coords_list_hi = gjson["features"][list_ind_hi]["geometry"]["coordinates"]
for ind_isl, island in enumerate(coords_list_hi):
    for ind_pnt, point in enumerate(island[0]):
        gjson["features"][list_ind_hi]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][0] = point[0] + incr_pct_hi * (point[0] +
                                                              min_abs_lon_hi)
        gjson["features"][list_ind_hi]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][1] = point[1] + incr_pct_hi * (point[1] -
                                                              min_lat_hi)

# Move Hawaii closer to the mainland such that the minimum minimum absolute
# longitude and lattitude are (-125, 27)
min_lat_hi_new = 27.5
min_abs_lon_hi_new = 124.5
for ind_isl, island in enumerate(coords_list):
    for ind_pnt, point in enumerate(island[0]):
        gjson["features"][list_ind_hi]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][0] = point[0] + (min_abs_lon_hi -
                                                min_abs_lon_hi_new)
        gjson["features"][list_ind_hi]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][1] = point[1] - (min_lat_hi - min_lat_hi_new)

# Add a state box around Delaware abbreviation DE
st_list_num = 2
de_coord_list = [gjson["features"][st_list_num]["geometry"]["coordinates"]]
new_box_de = [[
    [-75.4, 38.8],
    [-72.4, 38.3],
    [-72.4, 38.9],
    [-70.2, 38.9],
    [-70.2, 37.7],
    [-72.4, 37.7],
    [-72.4, 38.3]
]]
de_coord_list.append(new_box_de)
gjson["features"][st_list_num]["geometry"]["coordinates"] = de_coord_list
gjson["features"][st_list_num]["geometry"]["type"] = "MultiPolygon"

# Add a state box around Washington, DC (District of Columbia) abbreviation DC
st_list_num = 35
dc_coord_list = [gjson["features"][st_list_num]["geometry"]["coordinates"]]
new_box_dc = [[
    [-77.0, 38.9],
    [-73.3, 35.5],
    [-73.3, 36.1],
    [-71.1, 36.1],
    [-71.1, 34.9],
    [-73.3, 34.9],
    [-73.3, 35.5]
]]
dc_coord_list.append(new_box_dc)
gjson["features"][st_list_num]["geometry"]["coordinates"] = dc_coord_list
gjson["features"][st_list_num]["geometry"]["type"] = "MultiPolygon"

# Add a state box around Massachusetts abbreviation MD
st_list_num = 0
md_coord_list = gjson["features"][st_list_num]["geometry"]["coordinates"]
new_box_md = [[
    [-76.8, 39.3],
    [-72.7, 37.0],
    [-72.7, 37.6],
    [-70.5, 37.6],
    [-70.5, 36.4],
    [-72.7, 36.4],
    [-72.7, 37.0]
]]
md_coord_list.append(new_box_md)
gjson["features"][st_list_num]["geometry"]["coordinates"] = md_coord_list

# Add a state box around Massachusetts abbreviation MA
st_list_num = 29
ma_coord_list = gjson["features"][st_list_num]["geometry"]["coordinates"]
new_box_ma = [[
    [-71.7, 42.2],
    [-68.5, 42.2],
    [-68.5, 42.8],
    [-66.3, 42.8],
    [-66.3, 41.6],
    [-68.5, 41.6],
    [-68.5, 42.2]
]]
ma_coord_list.append(new_box_ma)
gjson["features"][st_list_num]["geometry"]["coordinates"] = ma_coord_list

# Add a state box around New Jersey abbreviation NJ
st_list_num = 34
nj_coord_list = [gjson["features"][st_list_num]["geometry"]["coordinates"]]
new_box_nj = [[
    [-74.4, 40.1],
    [-72.0, 39.7],
    [-72.0, 40.3],
    [-69.8, 40.3],
    [-69.8, 39.1],
    [-72.0, 39.1],
    [-72.0, 39.7]
]]
nj_coord_list.append(new_box_nj)
gjson["features"][st_list_num]["geometry"]["coordinates"] = nj_coord_list
gjson["features"][st_list_num]["geometry"]["type"] = "MultiPolygon"

# Add a state box around Rhode Island abbreviation RI
st_list_num = 50
ri_coord_list = gjson["features"][st_list_num]["geometry"]["coordinates"]
new_box_ri = [[
    [-71.5, 41.7],
    [-69.5, 40.4],
    [-69.5, 41.0],
    [-67.3, 41.0],
    [-67.3, 39.8],
    [-69.5, 39.8],
    [-69.5, 40.4]
]]
ri_coord_list.append(new_box_ri)
gjson["features"][st_list_num]["geometry"]["coordinates"] = ri_coord_list

# Merge the state tax type data into gjson for each state
state_taxtype_df
for ind_st, state in enumerate(gjson["features"]):
    st_abbrev = state["properties"]["STUSPS"]
    state["properties"]["rates"] = state_taxtype_df[
        state_taxtype_df["Abbrev"]==st_abbrev]["rates"].iloc[0]
    state["properties"]["tax_type"] = state_taxtype_df[
        state_taxtype_df["Abbrev"]==st_abbrev
    ]["TaxType_str_short"].iloc[0]

tax_type_labels = [
    'No income tax', 'Flat income tax rate', 'Nearly flat income tax rates',
    'Progressive income tax rates'
]
tax_type_colors = ["white", "red", "purple", "blue"]

source_shapes = {}
for category in tax_type_labels:
    source_shapes[category] = {"type": "FeatureCollection", "features": []}

for item in gjson["features"]:
    source_shapes[item["properties"]["tax_type"]]['features'].append(item)

TOOLS = "pan, box_zoom, wheel_zoom, hover, save, reset, help"

fig1 = figure(
    title=fig1_title,
    height=500,
    width=1050,
    tools=TOOLS,
    # tooltips=[
    #     ("State", @state_names), ("Tax type", @tax_type)
    # ]
    # match_aspect = True,
    min_border = 0,
    x_axis_location = None, y_axis_location = None,
    toolbar_location="right"
)
fig1.toolbar.logo = None
fig1.grid.grid_line_color = None

cmap = CategoricalColorMapper(
    palette=tax_type_colors, factors=tax_type_labels
)
for category in tax_type_labels:
    source_shape_1 = GeoJSONDataSource(
        geojson = json.dumps(source_shapes[category])
    )
    fig1.patches(
        'xs', 'ys', source=source_shape_1, fill_alpha=0.7,
        fill_color = {'field': 'tax_type', 'transform': cmap},
        line_color ='black', line_width=1.0, line_alpha=0.3,
        hover_line_color="black", hover_line_width=3.0, legend_label=category)
    
    hover = fig1.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = [
        ("State", "@NAME"),
        ("Tax type", "@tax_type"),
        ("Tax rate(s)", "@rates")
    ]

# Add 2-letter state abbreviation labels. See Bokeh documentation for labels at
# https://docs.bokeh.org/en/latest/docs/user_guide/basic/annotations.html.
label_lon_lat =[
    ["AL",  -87.40, 32.10],
    ["AK", -135.00, 48.30],
    ["AZ", -112.40, 34.00],
    ["AR",  -93.20, 34.30],
    ["CA", -121.00, 37.00],
    ["CO", -106.50, 38.50],
    ["CT",  -73.40, 41.15],
    ["DE",  -72.10, 37.80],
    ["DC",  -73.00, 35.00],
    ["FL",  -82.40, 28.00],
    ["GA",  -84.20, 32.20],
    ["HI", -125.00, 29.00],
    ["ID", -115.00, 43.00],
    ["IL",  -89.60, 39.50],
    ["IN",  -86.90, 39.60],
    ["IA",  -94.40, 41.60],
    ["KS",  -99.50, 38.00],
    ["KY",  -86.00, 37.00],
    ["LA",  -93.00, 30.20],
    ["ME",  -70.00, 44.50],
    ["MD",  -72.40, 36.50],
    ["MA",  -68.20, 41.80],
    ["MI",  -85.40, 42.70],
    ["MN",  -95.30, 45.50],
    ["MS",  -90.60, 32.10],
    ["MO",  -93.50, 38.00],
    ["MT", -110.50, 46.50],
    ["NE", -100.50, 41.00],
    ["NV", -118.00, 39.00],
    ["NH",  -72.45, 42.70],
    ["NM", -107.00, 34.00],
    ["NJ",  -71.60, 39.20],
    ["NY",  -76.00, 42.50],
    ["NC",  -79.20, 35.00],
    ["ND", -101.50, 46.80],
    ["OH",  -83.80, 39.90],
    ["OK",  -98.00, 35.00],
    ["OR", -121.50, 43.50],
    ["PA",  -78.50, 40.40],
    ["RI",  -69.00, 39.90],
    ["SC",  -81.50, 33.30],
    ["SD", -101.00, 44.00],
    ["TN",  -87.10, 35.40],
    ["TX", -100.00, 31.00],
    ["UT", -112.50, 39.00],
    ["VT",  -73.20, 44.10],
    ["VA",  -79.00, 37.10],
    ["WA", -121.00, 47.00],
    ["WV",  -81.90, 38.00],
    ["WI",  -90.40, 44.00],
    ["WY", -108.50, 42.50]
]
label_abbrev = [state[0] for state in label_lon_lat]
label_lon = [state[1] for state in label_lon_lat]
label_lat = [state[2] for state in label_lon_lat]
state_cds = ColumnDataSource(data=dict(
    lon=label_lon,
    lat=label_lat,
    abbrev=label_abbrev
))

state_labels = LabelSet(
    x='lon', y='lat', text='abbrev', text_font_size="9pt",
    text_font_style="bold", x_offset=0, y_offset=0, source=state_cds
)

fig1.add_layout(state_labels)

# Legend properties
fig1.legend.click_policy = 'mute'
fig1.legend.location = "center_left"

fig1.add_layout(
    Title(
        text="  Source: Richard W. Evans (@RickEcon), updated June 8, 2023.",
        align="left",
        text_font_size="3mm",
        text_font_style="italic",
    ),
    "below"
)
show(fig1)

Loading BokehJS ...

In [11]:
gjson

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'STATEFP': '24',
    'STATENS': '01714934',
    'AFFGEOID': '0400000US24',
    'GEOID': '24',
    'STUSPS': 'MD',
    'NAME': 'Maryland',
    'LSAD': '00',
    'ALAND': 25151100280,
    'AWATER': 6979966958,
    'rates': ['2.00%',
     '3.00%',
     '4.00%',
     '4.75%',
     '5.00%',
     '5.25%',
     '5.50%',
     '5.75%'],
    'tax_type': 'Progressive income tax rates'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-76.046213, 38.025532999999996],
       [-76.00733699999999, 38.036705999999995],
       [-75.98008899999999, 38.004891],
       [-75.98464799999999, 37.938120999999995],
       [-76.04652999999999, 37.953586],
       [-76.046213, 38.025532999999996]]],
     [[[-79.484372, 39.344300000000004],
       [-79.482366, 39.531689],
       [-79.47666199999999, 39.721078],
       [-79.3924584050224, 39.7214393586401],
       [-79.045576, 39.722927999999996],
       [

### 1.2. Figure 2. Rainy Day Fund and Total Reserves as Percentages of General-Fund Expenditures, Kansas and 50-State Median: 2000-2023

Get the Pew state rainy day fund data. This does not include District of Columbia. The 2022 data is now final. The 2023 data is labeled as "Enacted" which means projected based on enacted legislation. The Kanasas Rainy Day fund was established May 17, 2016, with the enactment of House Bill 2739. This made Kansas the 47th state with a Rainy Day Fund. See Robert Zahradnik and Steve Bailey, "[Kansas Becomes the 47th State to Create a Rainy Day Fund](https://www.pewtrusts.org/en/research-and-analysis/articles/2016/05/18/kansas-becomes-the-47th-state-to-create-a-rainy-day-fund), Article, Pew Charitable Trusts, May 18, 2016. For this reason, the Pew data for the Kansas Rainy Day fund start in 2020.

Note that I needed to fix New Jersey's 2022 and 2023 Rainy Day fund amounts in millions of dollars. In the dataset, the value is NaN. But it should be zero in 2022 and 2023. See John Reitmeyer, [Billions more int taxes came in, but state drained its 'rainy day' fund](https://www.njspotlightnews.org/2021/07/rainy-day-fund-drained-2-billion-general-fund-transferred/)," NJ Spotlight News, July 1, 2021.

In [15]:
# Read in the Pew data on rainy day funds by year and state from worksheet
rain_totbal_path = (
    "https://github.com/TheCGO/KS-FlatTax/blob/main/data/" +
    "ReservesBalancesData.xlsx?raw=true"
)
rain_df = pd.read_excel(
    rain_totbal_path,
    sheet_name="Rainy Day Fund Data",
    header=5,
    index_col=0,
    skipfooter=2
)
rain_df.replace(0, np.nan, inplace=True)

# Create a DataFrame of just the rainy day funds in $millions
# by state and by year
rain_dol_df = rain_df.loc[:'Wyoming', 'FY 2000.1':'FY 2023 Enacted']
rain_dol_df.rename(columns = {
    'FY 2000.1': '2000',
    'FY 2001.1': '2001',
    'FY 2002.1': '2002',
    'FY 2003.1': '2003',
    'FY 2004.1': '2004',
    'FY 2005.1': '2005',
    'FY 2006.1': '2006',
    'FY 2007.1': '2007',
    'FY 2008.1': '2008',
    'FY 2009.1': '2009',
    'FY 2010.1': '2010',
    'FY 2011.1': '2011',
    'FY 2012.1': '2012',
    'FY 2013.1': '2013',
    'FY 2014.1': '2014',
    'FY 2015.1': '2015',
    ' FY 2016.1': '2016',
    ' FY 2017.1': '2017',
    ' FY 2018.1': '2018',
    'FY 2019.1': '2019',
    'FY 2020.1': '2020',
    'FY 2021.1': '2021',
    'FY2022.1': '2022',
    'FY 2023 Enacted': '2023'
}, inplace = True)

# Create a DataFrame of just the rainy day funds as a percent of
# general fund expenditures by state and by year
rain_pct_df = rain_df.loc[:, 'FY 2000.2':'FY 2023 Enacted.1']
rain_pct_df.rename(columns = {
    'FY 2000.2': '2000',
    'FY 2001.2': '2001',
    'FY 2002.2': '2002',
    'FY 2003.2': '2003',
    'FY 2004.2': '2004',
    'FY 2005.2': '2005',
    'FY 2006.2': '2006',
    'FY 2007.2': '2007',
    'FY 2008.2': '2008',
    'FY 2009.2': '2009',
    'FY 2010.2': '2010',
    'FY 2011.2': '2011',
    'FY 2012.2': '2012',
    'FY 2013.2': '2013',
    'FY 2014.2': '2014',
    'FY 2015.2': '2015',
    ' FY 2016.2': '2016',
    ' FY 2017.2': '2017',
    ' FY 2018.2': '2018',
    'FY 2019.2': '2019',
    'FY 2020.2': '2020',
    'FY 2021.2': '2021',
    'FY2022.2': '2022',
    'FY 2023 Enacted.1': '2023'
}, inplace = True)

# Replace New Jersey NaN values with zeros in 2022 and 2023
rain_dol_df[["2022", "2023"]] = \
    rain_dol_df[["2022", "2023"]].replace(np.nan, 0.0)
rain_pct_df[["2022", "2023"]] = \
    rain_pct_df[["2022", "2023"]].replace(np.nan, 0.0)

In [16]:
# Read in the PEW data on total reserves and balances by year and state from
# worksheet
totbal_df = pd.read_excel(
    rain_totbal_path,
    sheet_name="Total Balances Data",
    header=5,
    index_col=0,
    skipfooter=2
)
totbal_df.replace(0, np.nan, inplace=True)

# Create a DataFrame of just the total reserves and balances funds in $millions
# by state and by year
totbal_dol_df = totbal_df.loc[:'Wyoming', 'FY 2000.1':'FY 2023 Enacted.1']
totbal_dol_df.rename(columns = {
    'FY 2000.1': '2000',
    'FY 2001.1': '2001',
    'FY 2002.1': '2002',
    'FY 2003.1': '2003',
    'FY 2004.1': '2004',
    'FY 2005.1': '2005',
    'FY 2006.1': '2006',
    'FY 2007.1': '2007',
    'FY 2008.1': '2008',
    'FY 2009.1': '2009',
    'FY 2010.1': '2010',
    'FY 2011.1': '2011',
    'FY 2012.1': '2012',
    'FY 2013.1': '2013',
    'FY 2014.1': '2014',
    'FY 2015.1': '2015',
    ' FY 2016.1': '2016',
    ' FY 2017.1': '2017',
    ' FY 2018.1': '2018',
    'FY 2019.1': '2019',
    'FY 2020.1': '2020',
    'FY 2021.1': '2021',
    'FY2022.1': '2022',
    'FY 2023 Enacted.1': '2023'
}, inplace = True)

# Create a DataFrame of just the total reserves and balances funds as percent of
# general fund expenditures by state and by year
totbal_pct_df = totbal_df.loc[:, 'FY 2000.2':'FY 2023 Enacted.2']
totbal_pct_df.rename(columns = {
    'FY 2000.2': '2000',
    'FY 2001.2': '2001',
    'FY 2002.2': '2002',
    'FY 2003.2': '2003',
    'FY 2004.2': '2004',
    'FY 2005.2': '2005',
    'FY 2006.2': '2006',
    'FY 2007.2': '2007',
    'FY 2008.2': '2008',
    'FY 2009.2': '2009',
    'FY 2010.2': '2010',
    'FY 2011.2': '2011',
    'FY 2012.2': '2012',
    'FY 2013.2': '2013',
    'FY 2014.2': '2014',
    'FY 2015.2': '2015',
    ' FY 2016.2': '2016',
    ' FY 2017.2': '2017',
    ' FY 2018.2': '2018',
    'FY 2019.2': '2019',
    'FY 2020.2': '2020',
    'FY 2021.2': '2021',
    'FY2022.2': '2022',
    'FY 2023 Enacted.2': '2023'
}, inplace = True)

Executing the cell below and the following cell will create the rain_totbal_50_ks_timeseries.html file and save it to the `./images/` directory in the repository. I created the .png version of the file by just screenshotting the `.html` image. These cells also save the source data used to create the image as `./data/fig2_source.csv`.

In [17]:
rain_pct_50_df = rain_pct_df.loc["50-state median", :].to_frame().reset_index()
rain_pct_50_df.rename(
    columns = {"index":"year", "50-state median":"fraction"}, inplace = True
)
rain_pct_50_df["percent"] = 100 * rain_pct_50_df["fraction"]

rain_pct_ks_df = rain_pct_df.loc["Kansas", :].to_frame().reset_index()
rain_pct_ks_df.rename(
    columns = {"index":"year", "Kansas":"fraction"}, inplace = True
)
rain_pct_ks_df["percent"] = 100 * rain_pct_ks_df["fraction"]

totbal_pct_50_df = \
    totbal_pct_df.loc["50-state median", :].to_frame().reset_index()
totbal_pct_50_df.rename(
    columns = {"index":"year", "50-state median":"fraction"}, inplace = True
)
totbal_pct_50_df["percent"] = 100 * totbal_pct_50_df["fraction"]

totbal_pct_ks_df = totbal_pct_df.loc["Kansas", :].to_frame().reset_index()
totbal_pct_ks_df.rename(
    columns = {'index':'year', "Kansas":"fraction"}, inplace = True
)
totbal_pct_ks_df["percent"] = 100 * totbal_pct_ks_df["fraction"]

# Merge the four DataFrames to save the joint dataframe as a .csv
fig2_source_df = rain_pct_50_df[['year', 'percent']].rename(
    columns={'percent':'rain_50m_pct'}
)
fig2_source_df = pd.merge(
    fig2_source_df,
    rain_pct_ks_df[['year', 'percent']].rename(
        columns={'percent':'rain_ks_pct'}
    ),
    on='year', how='inner'
)
fig2_source_df = pd.merge(
    fig2_source_df,
    totbal_pct_50_df[['year', 'percent']].rename(
        columns={'percent':'totbal_50m_pct'}
    ),
    on='year', how='inner'
)
fig2_source_df = pd.merge(
    fig2_source_df,
    totbal_pct_ks_df[['year', 'percent']].rename(
        columns={'percent':'totbal_ks_pct'}
    ),
    on='year', how='inner'
)
fig2_source_df.to_csv('./data/fig2_source.csv', index=False)
fig2_source_df

,year,rain_50m_pct,rain_ks_pct,totbal_50m_pct,totbal_ks_pct
0,2000,4.127259,NaN,8.766520,8.654639
1,2001,4.614270,NaN,7.148315,8.255824
2,2002,1.655668,NaN,2.887308,0.270930
3,2003,0.722015,NaN,2.656212,2.965559
4,2004,1.844196,NaN,5.246650,7.587166
5,2005,2.459819,NaN,9.016673,10.206605
6,2006,4.553273,NaN,11.801597,14.274318
7,2007,4.738918,NaN,11.327766,16.673503
8,2008,4.834502,NaN,8.292193,8.630240
9,2009,2.728263,NaN,4.582794,0.819550


In [18]:
# fig2_title = ("Rainy Day fund and total reserves as a percentage of " +
#               "general fund expenditures: 2000-2023")
fig2_title = ""
output_file(
    "./images/rain_totbal_50_ks_timeseries.html", title=fig2_title,
    mode='inline'
)
output_notebook()
min_year = 2000
max_year = 2023
min_pct = 2.0
max_pct = 0.0
pct_buffer = 0.05

var_list = [rain_pct_50_df, rain_pct_ks_df, totbal_pct_50_df, totbal_pct_ks_df]
color_list = ["#3477A5", "#D5AB53", "#3477A5", "#D5AB53"]
marker_list = ["circle", "circle", "square", "square"]
legend_label_list = [
    "Rainy Day Fund, 50-state median",
    "Rainy Day Fund, Kansas",
    "Total balances, 50-state median",
    "Total balances, Kansas"
]
cds_list = []

for k, df in enumerate(var_list):
    min_pct = np.minimum(min_pct, df["percent"].min())
    max_pct = np.maximum(max_pct, df["percent"].max())
    cds_list.append(ColumnDataSource(df))

fig2 = figure(title=fig2_title,
              height=420,
              width=700,
              x_axis_label='Year',
              x_range=(min_year - 1, max_year + 1.5),
              y_axis_label='Percent of general fund expenditures',
              y_range=(min_pct - pct_buffer * (max_pct - min_pct),
                       max_pct + pct_buffer * (max_pct - min_pct)),
              toolbar_location=None)

# Turn off any pan, drag, or scrolling ability
fig2.toolbar.active_drag = None
fig2.toolbar.active_scroll = None
fig2.toolbar.active_tap = None

# Set title font size and axes font sizes
fig2.title.text_font_size = '0pt'  # Hide title by setting font size to 0
fig2.title.text_color = '#434244'
fig2.xaxis.axis_label_text_font_size = '12pt'
fig2.xaxis.major_label_text_font_size = '12pt'
fig2.xaxis.major_label_text_color = '#434244'
fig2.yaxis.axis_label_text_font_size = '12pt'
fig2.yaxis.major_label_text_font_size = '12pt'
fig2.yaxis.major_label_text_color = '#434244'

# Modify tick intervals for X-axis and Y-axis
fig2.xaxis.ticker = SingleIntervalTicker(interval=2, num_minor_ticks=2)
fig2.xaxis.axis_line_color = '#434244'
fig2.xaxis.major_tick_line_color = '#434244'
fig2.xaxis.minor_tick_line_color = '#434244'
fig2.xgrid.ticker = SingleIntervalTicker(interval=2)
fig2.yaxis.ticker = SingleIntervalTicker(interval=10, num_minor_ticks=5)
fig2.yaxis.axis_line_color = '#434244'
fig2.yaxis.major_tick_line_color = '#434244'
fig2.yaxis.minor_tick_line_color = '#434244'
fig2.ygrid.ticker = SingleIntervalTicker(interval=10)

# Create lines and markers for time series
for k, yvar in enumerate(var_list):
    fig2.line(x='year', y='percent', source=cds_list[k], color=color_list[k],
              line_width=4, alpha=0.7)
    fig2.scatter(x='year', y='percent', source=cds_list[k], size=10,
                 line_width=1, line_color='black', fill_color=color_list[k],
                 marker=marker_list[k], line_alpha=0.7, fill_alpha=0.7,
                 legend_label=legend_label_list[k])

fig2.segment(x0=2022.4, y0=min_pct - pct_buffer * (max_pct - min_pct),
             x1=2022.4, y1=max_pct + pct_buffer * (max_pct - min_pct),
             color='#434244', line_dash='6 2', line_width=2)
    
label_temp = Label(
    x=2022.5, y=17.0, x_units='data', y_units='data', text='Projected',
    text_font_size='9pt', text_color='#434244'
)
fig2.add_layout(label_temp)

# Add information on hover
tooltips = [('Year', '@year'),
            ('Pct of gen. fund exps.','@percent{0.0}' + '%')]
fig2.add_tools(HoverTool(tooltips=tooltips, toggleable=False))

# Add legend
fig2.legend.location = 'top_center'
fig2.legend.border_line_width = 1
fig2.legend.border_line_color = '#434244'
fig2.legend.border_line_alpha = 1
fig2.legend.label_text_font_size = '12pt'
fig2.legend.label_text_color = '#434244'

# Add notes below image
note_text_list2 = [
    (
        'Source: Pew Charitable Trusts, "Fiscal 50: State Trends and Analysis," ' +
        'May 17, 2023,'
    ),
    ('        accessed May 26, 2023.')    
]
for note_text in note_text_list2:
    caption = Title(
        text=note_text, align='left', text_font_size='11pt',
        text_font_style='normal',
        text_color='#434244',
        # text_font='Open Sans'
    )
    fig2.add_layout(caption, 'below')

show(fig2)
# # This export_png() function requires selenium package as well as firefox
# # and geckodriver packages
# # (see https://docs.bokeh.org/en/3.0.3/docs/user_guide/output/export.html)
# export_png(fig, filename="/images/rain_totbal_50_tn_timeseries.png")

Loading BokehJS ...

Create Figure 2 without the source text underneath (for web publication).

In [19]:
output_file(
    "./images/rain_totbal_50_ks_timeseries_nosubtxt.html", title=fig2_title,
    mode='inline'
)
output_notebook()
min_year = 2000
max_year = 2023
min_pct = 2.0
max_pct = 0.0
pct_buffer = 0.05

var_list = [rain_pct_50_df, rain_pct_ks_df, totbal_pct_50_df, totbal_pct_ks_df]
color_list = ["#3477A5", "#D5AB53", "#3477A5", "#D5AB53"]
marker_list = ["circle", "circle", "square", "square"]
legend_label_list = [
    "Rainy Day Fund, 50-state median",
    "Rainy Day Fund, Kansas",
    "Total balances, 50-state median",
    "Total balances, Kansas"
]
cds_list = []

for k, df in enumerate(var_list):
    min_pct = np.minimum(min_pct, df["percent"].min())
    max_pct = np.maximum(max_pct, df["percent"].max())
    cds_list.append(ColumnDataSource(df))

fig2_nosubtxt = figure(title=fig2_title,
                       height=420,
                       width=700,
                       x_axis_label='Year',
                       x_range=(min_year - 1, max_year + 1.5),
                       y_axis_label='Percent of general fund expenditures',
                       y_range=(min_pct - pct_buffer * (max_pct - min_pct),
                                max_pct + pct_buffer * (max_pct - min_pct)),
                       toolbar_location=None)

# Turn off any pan, drag, or scrolling ability
fig2_nosubtxt.toolbar.active_drag = None
fig2_nosubtxt.toolbar.active_scroll = None
fig2_nosubtxt.toolbar.active_tap = None

# Set title font size and axes font sizes
fig2_nosubtxt.title.text_font_size = '0pt'  # Hide title by setting font size=0
fig2_nosubtxt.title.text_color = '#434244'
fig2_nosubtxt.xaxis.axis_label_text_font_size = '12pt'
fig2_nosubtxt.xaxis.major_label_text_font_size = '12pt'
fig2_nosubtxt.xaxis.major_label_text_color = '#434244'
fig2_nosubtxt.yaxis.axis_label_text_font_size = '12pt'
fig2_nosubtxt.yaxis.major_label_text_font_size = '12pt'
fig2_nosubtxt.yaxis.major_label_text_color = '#434244'

# Modify tick intervals for X-axis and Y-axis
fig2_nosubtxt.xaxis.ticker = SingleIntervalTicker(interval=2,
                                                  num_minor_ticks=2)
fig2_nosubtxt.xaxis.axis_line_color = '#434244'
fig2_nosubtxt.xaxis.major_tick_line_color = '#434244'
fig2_nosubtxt.xaxis.minor_tick_line_color = '#434244'
fig2_nosubtxt.xgrid.ticker = SingleIntervalTicker(interval=2)
fig2_nosubtxt.yaxis.ticker = SingleIntervalTicker(interval=10,
                                                  num_minor_ticks=5)
fig2_nosubtxt.yaxis.axis_line_color = '#434244'
fig2_nosubtxt.yaxis.major_tick_line_color = '#434244'
fig2_nosubtxt.yaxis.minor_tick_line_color = '#434244'
fig2_nosubtxt.ygrid.ticker = SingleIntervalTicker(interval=10)

# Create lines and markers for time series
for k, yvar in enumerate(var_list):
    fig2_nosubtxt.line(
        x='year', y='percent', source=cds_list[k], color=color_list[k],
        line_width=4, alpha=0.7
    )
    fig2_nosubtxt.scatter(
        x='year', y='percent', source=cds_list[k], size=10, line_width=1,
        line_color='black', fill_color=color_list[k], marker=marker_list[k],
        line_alpha=0.7, fill_alpha=0.7, legend_label=legend_label_list[k]
    )

fig2_nosubtxt.segment(x0=2022.4, y0=min_pct - pct_buffer * (max_pct - min_pct),
                      x1=2022.4, y1=max_pct + pct_buffer * (max_pct - min_pct),
                      color='#434244', line_dash='6 2', line_width=2)
    
label_temp = Label(
    x=2022.5, y=16.0, x_units='data', y_units='data', text='Projected',
    text_font_size='9pt', text_color='#434244'
)
fig2_nosubtxt.add_layout(label_temp)

# Add information on hover
tooltips = [('Year', '@year'),
            ('Pct of gen. fund exps.','@percent{0.0}' + '%')]
fig2_nosubtxt.add_tools(HoverTool(tooltips=tooltips, toggleable=False))

# Add legend
fig2_nosubtxt.legend.location = 'top_center'
fig2_nosubtxt.legend.border_line_width = 1
fig2_nosubtxt.legend.border_line_color = '#434244'
fig2_nosubtxt.legend.border_line_alpha = 1
fig2_nosubtxt.legend.label_text_font_size = '12pt'
fig2_nosubtxt.legend.label_text_color = '#434244'

# # Add notes below image
# note_text_list2 = [
#     (
#         'Source: Pew Charitable Trusts, "Fiscal 50: State Trends and Analysis," ' +
#         'May 17, 2023,'
#     ),
#     ('        accessed May 26, 2023.')    
# ]
# for note_text in note_text_list2:
#     caption = Title(
#         text=note_text, align='left', text_font_size='11pt',
#         text_font_style='normal',
#         text_color='#434244',
#         # text_font='Open Sans'
#     )
#     fig2.add_layout(caption, 'below')

show(fig2_nosubtxt)

# # This export_png() function requires selenium package as well as firefox
# # and geckodriver packages
# # (see https://docs.bokeh.org/en/3.0.3/docs/user_guide/output/export.html)
# export_png(fig, filename="/images/rain_totbal_50_tn_timeseries.png")

Loading BokehJS ...

### 1.3. Figure 3. Estimated 2022 Rainy Day Fund Balances and Total Reserves and Balances as Percentages of General-Fund Expenditures

In [20]:
rain_pct_2022_df = rain_pct_df.loc[:'Wyoming', '2022'].to_frame().reset_index()
rain_pct_2022_df.rename(
    columns = {"index": "state", "2022": "rain_frac"}, inplace = True
)
rain_pct_2022_df["rain_pct"] = 100 * rain_pct_2022_df["rain_frac"]
rain_pct_2022_df.drop('rain_frac', axis=1, inplace=True)

rain_dol_2022_df = rain_dol_df.loc[:'Wyoming', '2022'].to_frame().reset_index()
rain_dol_2022_df.rename(
    columns = {"index": "state", "2022": "rain_dol"}, inplace = True
)

totbal_pct_2022_df = \
    totbal_pct_df.loc[:'Wyoming', '2022'].to_frame().reset_index()
totbal_pct_2022_df.rename(
    columns = {"index": "state", "2022": "totbal_frac"}, inplace = True
)
totbal_pct_2022_df["totbal_pct"] = 100 * totbal_pct_2022_df["totbal_frac"]
totbal_pct_2022_df.drop('totbal_frac', axis=1, inplace=True)

totbal_dol_2022_df = \
    totbal_dol_df.loc[:'Wyoming', '2022'].to_frame().reset_index()
totbal_dol_2022_df.rename(
    columns = {"index": "state", "2022": "totbal_dol"}, inplace = True
)

# Merge the four DataFrames
rain_totbal_pct_dol_2022_df = rain_pct_2022_df.copy()
rain_totbal_pct_dol_2022_df = pd.merge(
    rain_totbal_pct_dol_2022_df, rain_dol_2022_df, on='state', how='inner'
)
rain_totbal_pct_dol_2022_df["rain_color"] = "#D5AB53"
rain_totbal_pct_dol_2022_df["rain_color"][
    rain_totbal_pct_dol_2022_df["state"]=="Kansas"
] = "#EED17E"
rain_totbal_pct_dol_2022_df = pd.merge(
    rain_totbal_pct_dol_2022_df, totbal_pct_2022_df, on='state', how='inner'
)
rain_totbal_pct_dol_2022_df = pd.merge(
    rain_totbal_pct_dol_2022_df, totbal_dol_2022_df, on='state', how='inner'
)
rain_totbal_pct_dol_2022_df["totbal_color"] = "#3477A5"
rain_totbal_pct_dol_2022_df["totbal_color"][
    rain_totbal_pct_dol_2022_df["state"]=="Kansas"
] = "#74AAC7"

# Create a new variable that is totbal_pct - rain_pct
rain_totbal_pct_dol_2022_df["totbal_rain_pct_dif"] = (
    rain_totbal_pct_dol_2022_df["totbal_pct"] -
    rain_totbal_pct_dol_2022_df["rain_pct"]
)

# Sort by rainy day fund percent
rain_totbal_pct_dol_2022_sorted_df = rain_totbal_pct_dol_2022_df.sort_values(
    by=['rain_pct', 'totbal_pct'], ascending=[True, True]
)

rain_totbal_pct_dol_2022_sorted_df.to_csv('./data/fig3_source.csv',
                                          index=False)
rain_totbal_pct_dol_2022_sorted_df

,state,rain_pct,rain_dol,rain_color,totbal_pct,totbal_dol,totbal_color,totbal_rain_pct_dif
29,New Jersey,0.000000,0.000000,#D5AB53,14.086556,7360.000000,#3477A5,14.086556
46,Washington,1.106906,312.400000,#D5AB53,18.595958,5248.300000,#3477A5,17.489051
12,Illinois,1.654149,751.000000,#D5AB53,4.030748,1830.000000,#3477A5,2.376600
10,Hawaii,3.684520,325.800000,#D5AB53,33.303051,2944.789000,#3477A5,29.618531
31,New York,3.931626,3319.000000,#D5AB53,39.153972,33053.000000,#3477A5,35.222346
25,Montana,4.286700,118.000000,#D5AB53,71.653350,1972.401760,#3477A5,67.366649
38,Rhode Island,5.330159,279.113537,#D5AB53,9.466316,495.703197,#3477A5,4.136157
19,Maryland,5.477720,1161.200000,#D5AB53,31.416697,6659.900000,#3477A5,25.938977
7,Delaware,5.541826,280.300000,#D5AB53,51.511497,2605.400000,#3477A5,45.969671
17,Louisiana,6.855598,772.437585,#D5AB53,6.889988,776.312398,#3477A5,0.034390


In [21]:
# fig3_title = ('2022 Rainy day Fund Balances and Total Fund ' +
#               'Balances as Percent of General Fund Expenditures')
fig3_title = ""
output_file("./images/rain_totbal_pct_2022.html", title=fig3_title,
            mode='inline')
output_notebook()

states = rain_totbal_pct_dol_2022_sorted_df['state'].tolist()
legend_label_list = ["Rainy day fund", "Total reserves and balances"]
rain_totbal_pct_dol_2022_cds = \
    ColumnDataSource(rain_totbal_pct_dol_2022_sorted_df)

min_pct = rain_totbal_pct_dol_2022_sorted_df['rain_pct'].min()
max_pct = rain_totbal_pct_dol_2022_sorted_df['totbal_pct'].max()
pct_buffer = 0.05

fig3 = figure(title=fig3_title,
              height=900,
              width=550,
              y_range=states,
              x_axis_label='Percent of general fund expenditures',
              x_range=(min_pct - pct_buffer * (max_pct - min_pct),
                       max_pct + pct_buffer * (max_pct - min_pct)),
              toolbar_location=None)

# Turn off any pan, drag, or scrolling ability
fig3.toolbar.active_drag = None
fig3.toolbar.active_scroll = None
fig3.toolbar.active_tap = None

# Set title font size and axes font sizes
fig3.title.text_font_size = '0pt'  # Hide title by setting font size to 0
fig3.title.text_color = '#434244'
fig3.xaxis.axis_label_text_font_size = '11pt'
fig3.xaxis.major_label_text_font_size = '11pt'
fig3.xaxis.major_label_text_color = '#434244'
fig3.yaxis.axis_label_text_font_size = '9pt'
fig3.yaxis.major_label_text_font_size = '9pt'
fig3.yaxis.major_label_text_color = '#434244'

# Modify tick intervals for X-axis and Y-axis
fig3.xaxis.ticker = SingleIntervalTicker(interval=10, num_minor_ticks=5)
fig3.xaxis.axis_line_color = '#434244'
fig3.xaxis.major_tick_line_color = '#434244'
fig3.xaxis.minor_tick_line_color = '#434244'
fig3.xgrid.ticker = SingleIntervalTicker(interval=10)
fig3.yaxis.axis_line_color = '#434244'
fig3.yaxis.major_tick_line_color = '#434244'
fig3.yaxis.minor_tick_line_color = '#434244'

fig3.hbar_stack(
    ['rain_pct', 'totbal_rain_pct_dif'], y='state', height=0.9,
    color=['rain_color', 'totbal_color'],
    source=rain_totbal_pct_dol_2022_cds, legend_label=legend_label_list
)

# Add information on hover
tooltips = [('State', '@state'),
            ('Rainy day fund % exps.','@rain_pct{0.0}' + '%'),
            ('Total balances % exps.','@totbal_pct{0.0}' + '%'),
           ]
fig3.add_tools(HoverTool(tooltips=tooltips, toggleable=False))

# Add legend
fig3.legend.location = (210, 240)
fig3.legend.border_line_width = 1
fig3.legend.border_line_color = '#434244'
fig3.legend.border_line_alpha = 1
fig3.legend.label_text_color = '#434244'
fig3.legend.label_text_font_size = '11pt'
fig3.legend.label_text_color = '#434244'
fig3.y_range.range_padding = 0.02

# Add notes below image
note_text_list3 = [
    (
        'Source: Pew Charitable Trusts, "Fiscal 50: State Trends and'
    ),
    (
        '        Analysis," May 17, 2023, accessed May 26, 2023.'
    ),
    (
        'Note: For states in which the blue bar is not visible for total ' +
        'balances'
    ),
    (
        '         and reserves, the value equals the rainy day fund balance'
    ),
    (
        '         percentage.'
    )
]
for note_text in note_text_list3:
    caption = Title(text=note_text, align='left', text_font_size='11pt',
                    text_font_style='normal', text_color='#434244')
    fig3.add_layout(caption, 'below')

show(fig3)

Loading BokehJS ...

Create Figure 3 without the source text underneath (for web publication).

In [22]:
output_file(
    "./images/rain_totbal_pct_2022_nosubtxt.html", title=fig3_title,
    mode='inline'
)
output_notebook()

states = rain_totbal_pct_dol_2022_sorted_df['state'].tolist()
legend_label_list = ["Rainy day fund", "Total reserves and balances"]
rain_totbal_pct_dol_2022_cds = \
    ColumnDataSource(rain_totbal_pct_dol_2022_sorted_df)

min_pct = rain_totbal_pct_dol_2022_sorted_df['rain_pct'].min()
max_pct = rain_totbal_pct_dol_2022_sorted_df['totbal_pct'].max()
pct_buffer = 0.05

fig3_nosubtxt = figure(
    title=fig3_title,
    height=900,
    width=550,
    y_range=states,
    x_axis_label='Percent of general fund expenditures',
    x_range=(min_pct - pct_buffer * (max_pct - min_pct),
             max_pct + pct_buffer * (max_pct - min_pct)),
    toolbar_location=None
)

# Turn off any pan, drag, or scrolling ability
fig3_nosubtxt.toolbar.active_drag = None
fig3_nosubtxt.toolbar.active_scroll = None
fig3_nosubtxt.toolbar.active_tap = None

# Set title font size and axes font sizes
fig3_nosubtxt.title.text_font_size = '0pt'  # Hide title by setting font size to 0
fig3_nosubtxt.title.text_color = '#434244'
fig3_nosubtxt.xaxis.axis_label_text_font_size = '11pt'
fig3_nosubtxt.xaxis.major_label_text_font_size = '11pt'
fig3_nosubtxt.xaxis.major_label_text_color = '#434244'
fig3_nosubtxt.yaxis.axis_label_text_font_size = '9pt'
fig3_nosubtxt.yaxis.major_label_text_font_size = '9pt'
fig3_nosubtxt.yaxis.major_label_text_color = '#434244'

# Modify tick intervals for X-axis and Y-axis
fig3_nosubtxt.xaxis.ticker = SingleIntervalTicker(interval=10,
                                                  num_minor_ticks=5)
fig3_nosubtxt.xaxis.axis_line_color = '#434244'
fig3_nosubtxt.xaxis.major_tick_line_color = '#434244'
fig3_nosubtxt.xaxis.minor_tick_line_color = '#434244'
fig3_nosubtxt.xgrid.ticker = SingleIntervalTicker(interval=10)
fig3_nosubtxt.yaxis.axis_line_color = '#434244'
fig3_nosubtxt.yaxis.major_tick_line_color = '#434244'
fig3_nosubtxt.yaxis.minor_tick_line_color = '#434244'

fig3_nosubtxt.hbar_stack(
    ['rain_pct', 'totbal_rain_pct_dif'], y='state', height=0.9,
    color=['rain_color', 'totbal_color'],
    source=rain_totbal_pct_dol_2022_cds, legend_label=legend_label_list
)

# Add information on hover
tooltips = [('State', '@state'),
            ('Rainy day fund % exps.','@rain_pct{0.0}' + '%'),
            ('Total balances % exps.','@totbal_pct{0.0}' + '%'),
           ]
fig3_nosubtxt.add_tools(HoverTool(tooltips=tooltips, toggleable=False))

# Add legend
fig3_nosubtxt.legend.location = (210, 300)
fig3_nosubtxt.legend.border_line_width = 1
fig3_nosubtxt.legend.border_line_color = '#434244'
fig3_nosubtxt.legend.border_line_alpha = 1
fig3_nosubtxt.legend.label_text_color = '#434244'
fig3_nosubtxt.legend.label_text_font_size = '11pt'
fig3_nosubtxt.legend.label_text_color = '#434244'
fig3_nosubtxt.y_range.range_padding = 0.02

# # Add notes below image
# note_text_list3 = [
#     (
#         'Source: Pew Charitable Trusts, "Fiscal 50: State Trends and'
#     ),
#     (
#         '        Analysis," May 17, 2023, accessed May 26, 2023.'
#     ),
#     (
#         'Note: For states in which the blue bar is not visible for total ' +
#         'balances'
#     ),
#     (
#         '         and reserves, the value equals the rainy day fund balance'
#     ),
#     (
#         '         percentage.'
#     )
# ]
# for note_text in note_text_list3:
#     caption = Title(text=note_text, align='left', text_font_size='11pt',
#                     text_font_style='normal', text_color='#434244')
#     fig3.add_layout(caption, 'below')

show(fig3_nosubtxt)

Loading BokehJS ...

### 1.4. Table 1. Number of states for which 2022 amounts represent 23-year high for select categories of rainy day fund and total balances and reserves statistics: 2000-2022

In [23]:
# Number of states for which estimated 2022 rainy day fund balances
# represent a 23-year high
rain_dol_df["max_2021"] = False
rain_dol_df["max_2022"] = False
rain_dol_df["max_2021"][
    rain_dol_df["2021"] > rain_dol_df.loc[:, "2000":"2020"].max(axis=1)
] = True
rain_dol_df["max_2022"][
    rain_dol_df["2022"] > rain_dol_df.loc[:, "2000":"2021"].max(axis=1)
] = True
rain_dol_22high = rain_dol_df["max_2021"].sum()
rain_dol_23high = rain_dol_df["max_2022"].sum()
print("Number of states for which 2022 rainy day fund balances")
print("are 23-year high is", rain_dol_23high, "states.")
print("Number of states for which 2021 rainy day fund balances")
print("are 22-year high is", rain_dol_22high, "states.")
print("Kansas 2022:", rain_dol_df.loc["Kansas", "max_2022"])
print("Kansas 2021:", rain_dol_df.loc["Kansas", "max_2021"])

Number of states for which 2022 rainy day fund balances
are 23-year high is 37 states.
Number of states for which 2021 rainy day fund balances
are 22-year high is 28 states.
Kansas 2022: True
Kansas 2021: False


In [24]:
# Number of states for which estimated 2022 rainy day fund balances
# as percent of general fund expenditures represent a 23-year high
rain_pct_df["max_2021"] = False
rain_pct_df["max_2022"] = False
rain_pct_df["max_2021"][
    rain_pct_df["2021"] > rain_pct_df.loc[:, "2000":"2020"].max(axis=1)
] = True
rain_pct_df["max_2022"][
    rain_pct_df["2022"] > rain_pct_df.loc[:, "2000":"2021"].max(axis=1)
] = True
rain_pct_22high = rain_pct_df["max_2021"].sum()
rain_pct_23high = rain_pct_df["max_2022"].sum()
print("")
print("Number of states for which 2022 rainy day fund balances")
print("as percent of general fund expenditures are 23-year high")
print("is", rain_pct_23high, "states.")
print("Number of states for which 2021 rainy day fund balances")
print("as percent of general fund expenditures are 22-year high")
print("is", rain_pct_22high, "states.")
print("Kansas 2022:", rain_pct_df.loc["Kansas", "max_2022"])
print("Kansas 2021:", rain_pct_df.loc["Kansas", "max_2021"])


Number of states for which 2022 rainy day fund balances
as percent of general fund expenditures are 23-year high
is 18 states.
Number of states for which 2021 rainy day fund balances
as percent of general fund expenditures are 22-year high
is 24 states.
Kansas 2022: True
Kansas 2021: True


In [25]:
# Number of states for which estimated 2022 total balances and reserves
# represent a 23-year high
totbal_dol_df["max_2021"] = False
totbal_dol_df["max_2022"] = False
totbal_dol_df["max_2021"][
    totbal_dol_df["2021"] > totbal_dol_df.loc[:, "2000":"2020"].max(axis=1)
] = True
totbal_dol_df["max_2022"][
    totbal_dol_df["2022"] > totbal_dol_df.loc[:, "2000":"2021"].max(axis=1)
] = True
totbal_dol_22high = totbal_dol_df["max_2021"].sum()
totbal_dol_23high = totbal_dol_df["max_2022"].sum()
print("")
print("Number of states for which 2022 total balances and reserves")
print("are 23-year high is", totbal_dol_23high, "states.")
print("Number of states for which 2021 total balances and reserves")
print("are 22-year high is", totbal_dol_22high, "states.")
print("Kansas 2022:", totbal_dol_df.loc["Kansas", "max_2022"])
print("Kansas 2021:", totbal_dol_df.loc["Kansas", "max_2021"])


Number of states for which 2022 total balances and reserves
are 23-year high is 43 states.
Number of states for which 2021 total balances and reserves
are 22-year high is 40 states.
Kansas 2022: True
Kansas 2021: True


In [26]:
# Number of states for which estimated 2022 total balances and reserves
# as percent of general fund expenditures represent a 23-year high
totbal_pct_df["max_2021"] = False
totbal_pct_df["max_2022"] = False
totbal_pct_df["max_2021"][
    totbal_pct_df["2021"] > totbal_pct_df.loc[:, "2000":"2020"].max(axis=1)
] = True
totbal_pct_df["max_2022"][
    totbal_pct_df["2022"] > totbal_pct_df.loc[:, "2000":"2021"].max(axis=1)
] = True
totbal_pct_22high = totbal_pct_df["max_2021"].sum()
totbal_pct_23high = totbal_pct_df["max_2022"].sum()
print("")
print("Number of states for which 2022 total balances and reserves")
print("as percent of general fund expenditures are 23-year high")
print("is", totbal_pct_23high, "states.")
print("Number of states for which 2021 total balances and reserves")
print("as percent of general fund expenditures are 22-year high")
print("is", totbal_pct_22high, "states.")
print("Kansas 2022:", totbal_pct_df.loc["Kansas", "max_2022"])
print("Kansas 2021:", totbal_pct_df.loc["Kansas", "max_2021"])


Number of states for which 2022 total balances and reserves
as percent of general fund expenditures are 23-year high
is 37 states.
Number of states for which 2021 total balances and reserves
as percent of general fund expenditures are 22-year high
is 33 states.
Kansas 2022: True
Kansas 2021: True


## 2. Kansas Individual Income Tax Rate Landscape

## 3. Effects of different size rate cuts

In [19]:
from fiscalsim_us import Simulation
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
import pandas as pd

### 3.1. Define the three household types

#### 3.1.1 Define the low income household (household before-tax income = $25,000)

In [20]:
situation_low = {
    "people": {
        "you": {
            "age": {"2023": "38"},
            "charitable_cash_donations": {"2023": "0"},
            "employment_income": {"2023": "25000"},
            "medical_out_of_pocket_expenses": {"2023": "200"},
        },
        "your partner": {
            "age": {"2023": "35"},
            "employment_income": {"2023": "0"},
        },
        "your first dependent": {
            "age": {"2023": "10"},
            "is_tax_unit_dependent": {"2023": True},
        },
        "your second dependent": {
            "age": {"2023": "6"},
            "is_tax_unit_dependent": {"2023": True},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"],
            "marital_unit_id": {"2023": 0}
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2023": 2}
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2023": 3}
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "broadband_cost": {"2023": "600"},
            "childcare_expenses": {"2023": "0"},
            "housing_cost": {"2023": "8000"},
            "phone_cost": {"2023": "700"},
            "spm_unit_id": {"2023": "0"},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "state_living_arrangement": {"2023": "FULL_COST"},
            "state_name": {"2023": "UT"}
        }
    }
}

#### 3.1.2 Define the middle income household (household before-tax income = $80,000)

In [21]:
situation_mid = {
    "people": {
        "you": {
            "age": {"2023": "38"},
            "charitable_cash_donations": {"2023": "6000"},
            "employment_income": {"2023": "45000"},
            "medical_out_of_pocket_expenses": {"2023": "2000"},
        },
        "your partner": {
            "age": {"2023": "35"},
            "employment_income": {"2023": "35000"},
        },
        "your first dependent": {
            "age": {"2023": "10"},
            "is_tax_unit_dependent": {"2023": True},
        },
        "your second dependent": {
            "age": {"2023": "10"},
            "is_tax_unit_dependent": {"2023": True},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"],
            "marital_unit_id": {"2023": 0}
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2023": 2}
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2023": 3}
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "broadband_cost": {"2023": "900"},
            "childcare_expenses": {"2023": "1000"},
            "housing_cost": {"2023": "20000"},
            "phone_cost": {"2023": "1500"},
            "spm_unit_id": {"2023": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "state_living_arrangement": {"2023": "FULL_COST"},
            "state_name": {"2023": "UT"}
        }
    }
}

#### 3.1.3 Define the high income household (household before-tax income = $200,000)

In [22]:
situation_high = {
    "people": {
        "you": {
            "age": {"2023": "38"},
            "charitable_cash_donations": {"2023": "20000"},
            "employment_income": {"2023": "100000"},
            "medical_out_of_pocket_expenses": {"2023": "4000"},
        },
        "your partner": {
            "age": {"2023": "35"},
            "employment_income": {"2023": "100000"},
        },
        "your first dependent": {
            "age": {"2023": "10"},
            "is_tax_unit_dependent": {"2023": True},
        },
        "your second dependent": {
            "age": {"2023": "10"},
            "is_tax_unit_dependent": {"2023": True},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"],
            "marital_unit_id": {"2023": 0}
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2023": 2}
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2023": 3}
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "broadband_cost": {"2023": "1200"},
            "childcare_expenses": {"2023": "1500"},
            "housing_cost": {"2023": "36000"},
            "phone_cost": {"2023": "2400"},
            "spm_unit_id": {"2023": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "state_living_arrangement": {"2023": "FULL_COST"},
            "state_name": {"2023": "UT"}
        }
    }
}

### 3.2. Solve for the baseline

#### 3.2.1. Baseline simulation for the low income household

In [23]:
def modify_parameters_b(parameters):
    pass
    return parameters


class reform_b(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters_b)

simulation_b_l = Simulation(
    reform=reform_b,
    situation=situation_low,
)

simulation_b_l.trace = True
simulation_b_l.calculate("household_net_income", 2023)
simulation_b_l.tracer.print_computation_log()

  household_net_income<2023, (default)> = [42138.2]
    household_market_income<2023, (default)> = [25000.]
      employment_income<2023, (default)> = [25000.     0.     0.     0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
        long_t

#### 3.2.2. Baseline simulation for the middle income household

In [24]:
simulation_b_m = Simulation(
    reform=reform_b,
    situation=situation_mid,
)

simulation_b_m.trace = True
simulation_b_m.calculate("household_net_income", 2023)
simulation_b_m.tracer.print_computation_log()

  household_net_income<2023, (default)> = [69606.484]
    household_market_income<2023, (default)> = [80000.]
      employment_income<2023, (default)> = [45000. 35000.     0.     0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
        long

#### 3.2.3. Baseline simulation for the high income household

In [25]:
simulation_b_h = Simulation(
    reform=reform_b,
    situation=situation_high,
)

simulation_b_h.trace = True
simulation_b_h.calculate("household_net_income", 2023)
simulation_b_h.tracer.print_computation_log()

  household_net_income<2023, (default)> = [150505.48]
    household_market_income<2023, (default)> = [200000.]
      employment_income<2023, (default)> = [100000. 100000.      0.      0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
       

### 3.3. Effects on three household types of rate cut to 4.65%

#### 3.3.1. Effect of rate cut to 4.65% on low income household

In [26]:
def modify_parameters_1(parameters):
    parameters.gov.states.ut.tax.income.rate.update(
        start=instant("2022-01-01"),
        stop=instant("2028-12-31"),
        value=0.0465
    )
    return parameters

class reform_1(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters_1)

simulation_1_l = Simulation(
    reform=reform_1,
    situation=situation_low,
)

simulation_1_l.trace = True
simulation_1_l.calculate("household_net_income", 2023)
simulation_1_l.tracer.print_computation_log()

  household_net_income<2023, (default)> = [42138.2]
    household_market_income<2023, (default)> = [25000.]
      employment_income<2023, (default)> = [25000.     0.     0.     0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
        long_t

#### 3.3.2. Effect of rate cut to 4.65% on middle income household

In [27]:
simulation_1_m = Simulation(
    reform=reform_1,
    situation=situation_mid,
)

simulation_1_m.trace = True
simulation_1_m.calculate("household_net_income", 2023)
simulation_1_m.tracer.print_computation_log()

  household_net_income<2023, (default)> = [69766.484]
    household_market_income<2023, (default)> = [80000.]
      employment_income<2023, (default)> = [45000. 35000.     0.     0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
        long

#### 3.3.3. Effect of rate cut to 4.65% on high income household

In [28]:
simulation_1_h = Simulation(
    reform=reform_1,
    situation=situation_high,
)

simulation_1_h.trace = True
simulation_1_h.calculate("household_net_income", 2023)
simulation_1_h.tracer.print_computation_log()

  household_net_income<2023, (default)> = [150905.48]
    household_market_income<2023, (default)> = [200000.]
      employment_income<2023, (default)> = [100000. 100000.      0.      0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
       

### 3.4. Effects on three household types of rate cut to 4.50%

#### 3.4.1. Effect of rate cut to 4.50% on low income household

In [29]:
def modify_parameters_2(parameters):
    parameters.gov.states.ut.tax.income.rate.update(
        start=instant("2023-01-01"),
        stop=instant("2027-12-31"),
        value=0.045
    )

class reform_2(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters_2)

simulation_2_l = Simulation(
    reform=reform_2,
    situation=situation_low,
)

simulation_2_l.trace = True
simulation_2_l.calculate("household_net_income", 2023)
simulation_2_l.tracer.print_computation_log()

  household_net_income<2023, (default)> = [42138.2]
    household_market_income<2023, (default)> = [25000.]
      employment_income<2023, (default)> = [25000.     0.     0.     0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
        long_t

#### 3.4.2. Effect of rate cut to 4.50% on middle income household

In [30]:
simulation_2_m = Simulation(
    reform=reform_2,
    situation=situation_mid,
)

simulation_2_m.trace = True
simulation_2_m.calculate("household_net_income", 2023)
simulation_2_m.tracer.print_computation_log()

  household_net_income<2023, (default)> = [69886.484]
    household_market_income<2023, (default)> = [80000.]
      employment_income<2023, (default)> = [45000. 35000.     0.     0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
        long

#### 3.4.3. Effect of rate cut to 4.50% on high income household

In [31]:
simulation_2_h = Simulation(
    reform=reform_2,
    situation=situation_high,
)

simulation_2_h.trace = True
simulation_2_h.calculate("household_net_income", 2023)
simulation_2_h.tracer.print_computation_log()

  household_net_income<2023, (default)> = [151161.48]
    household_market_income<2023, (default)> = [200000.]
      employment_income<2023, (default)> = [100000. 100000.      0.      0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
       

### 3.5. Effects on three household types of rate cut to 4.00%

#### 3.5.1. Effect of rate cut to 4.00% on low income household

In [32]:
def modify_parameters_3(parameters):
    parameters.gov.states.ut.tax.income.rate.update(
        start=instant("2023-01-01"),
        stop=instant("2027-12-31"),
        value=0.04
    )

class reform_3(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters_3)

simulation_3_l = Simulation(
    reform=reform_3,
    situation=situation_low,
)

simulation_3_l.trace = True
simulation_3_l.calculate("household_net_income", 2023)
simulation_3_l.tracer.print_computation_log()

  household_net_income<2023, (default)> = [42138.2]
    household_market_income<2023, (default)> = [25000.]
      employment_income<2023, (default)> = [25000.     0.     0.     0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
        long_t

#### 3.5.2. Effect of rate cut to 4.00% on middle income household

In [33]:
simulation_3_m = Simulation(
    reform=reform_3,
    situation=situation_mid,
)

simulation_3_m.trace = True
simulation_3_m.calculate("household_net_income", 2023)
simulation_3_m.tracer.print_computation_log()

  household_net_income<2023, (default)> = [70286.484]
    household_market_income<2023, (default)> = [80000.]
      employment_income<2023, (default)> = [45000. 35000.     0.     0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
        long

#### 3.5.3. Effect of rate cut to 4.00% on high income household

In [34]:
simulation_3_h = Simulation(
    reform=reform_3,
    situation=situation_high,
)

simulation_3_h.trace = True
simulation_3_h.calculate("household_net_income", 2023)
simulation_3_h.tracer.print_computation_log()

  household_net_income<2023, (default)> = [152161.48]
    household_market_income<2023, (default)> = [200000.]
      employment_income<2023, (default)> = [100000. 100000.      0.      0.]
      self_employment_income<2023, (default)> = [0. 0. 0. 0.]
      pension_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_pension_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_pension_income<2023, (default)> = [0. 0. 0. 0.]
      dividend_income<2023, (default)> = [0. 0. 0. 0.]
        qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
        non_qualified_dividend_income<2023, (default)> = [0. 0. 0. 0.]
      interest_income<2023, (default)> = [0. 0. 0. 0.]
        tax_exempt_interest_income<2023, (default)> = [0. 0. 0. 0.]
        taxable_interest_income<2023, (default)> = [0. 0. 0. 0.]
      gi_cash_assistance<2023, (default)> = [0. 0. 0. 0.]
      capital_gains<2023, (default)> = [0. 0. 0. 0.]
        short_term_capital_gains<2023, (default)> = [0. 0. 0. 0.]
       